In [0]:
from pyspark.sql import functions as f
from delta.tables import *
from pyspark.sql.catalog import Catalog

In [0]:
sessao = '/Volumes/workspace/default/qa/votacao_secao_2022_ES.csv'

df = (
    spark
    .read
    .csv(
        sessao, 
        header=True, 
        inferSchema=True, 
        sep=';', 
        encoding='latin1'
    )
)

df.limit(10).display()

In [0]:
df.count()

In [0]:
print(df.columns)

In [0]:
df_dim_candidato = (
    df
    .select(
        ['NR_VOTAVEL', 'NM_VOTAVEL']
    )
    .distinct()
    .withColumn(
        'SQ_CANDIDATO', 
        f.md5(
            f.trim(
                f.concat(
                    'NR_VOTAVEL', 'NM_VOTAVEL'
                )
            )
        )
    )
)

display(df_dim_candidato.limit(10))

In [0]:
df_dim_candidato.count()

In [0]:
spark.sql('create database if not exists workspace.default')
# df_sessao.write.mode('overwrite').saveAsTable('workspace.default.votacao_secao')

In [0]:
catalog = spark.catalog

# Verifica se a tabela existe no catálogo e schema especificados
if catalog.tableExists(f"workspace.default.dim_candidato"):

    dim_candidato = DeltaTable.forName(spark, "workspace.default.dim_candidato")

    (    
        dim_candidato
        .alias('desttable')
        .merge(
            df_dim_candidato.alias('updates'), 
                'updates.SQ_CANDIDATO = desttable.SQ_CANDIDATO'
            )
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )

else:
    (
        df_dim_candidato
        .write
        .format('delta')
        .mode('overwrite')
        .saveAsTable('workspace.default.dim_candidato')
    )

In [0]:
%sql 
SELECT
  *
FROM 
  default.dim_candidato
LIMIT 10